ÖDEV METNİ  
Titanic verilerini Kaggle'dan indirin. Buradaki train.csv dosyasındaki veriler ihtiyacınızı görecektir.  
Verilerinizi eğitim ve test kümelerine ayırın.  
Modelinizi oluşturarak ayırdığınız test kümesindeki verilere göre hayatta kalma durumlarını tahmin edin  
Modelinizin performansı tatmin edici mi? Açıklayın.  
Bazı değişkenleri ekleyerek veya çıkararak modelinizi tahmin performansı açısından geliştirmeye çalışın.  
Lojistik Regresyon'un avantaj ve dezavantajlarını araştırın ve mentörünüzle tartışın.  

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df['Age']=df['Age'].interpolate()

In [7]:
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.025332,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.062164,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.304934,0.083081,0.018443,-0.549500
Age,0.025332,-0.062164,-0.304934,1.000000,-0.213410,-0.170013,0.087119
SibSp,-0.057527,-0.035322,0.083081,-0.213410,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.170013,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.087119,0.159651,0.216225,1.000000


In [8]:
#hayattta kalmayı etkileyeceğini düşündüğüm değişkenleri seçtim.
X=df[['Pclass','Sex','Age','SibSp','Parch']]
y=df['Survived']


In [9]:
#altta cinsiyetin dummy halini almalısın
X = pd.get_dummies(data=X, drop_first=True)

In [10]:
X_eğitim, X_test, y_eğitim, y_test =  train_test_split(X, y, test_size=0.20, random_state=111)

In [11]:
log_reg = LogisticRegression()

In [12]:
log_reg.fit(X_eğitim, y_eğitim)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
egitim_dogruluk = log_reg.score(X_eğitim, y_eğitim)
test_dogruluk = log_reg.score(X_test, y_test)
print('One-vs-rest', '-'*20, 
      'Modelin eğitim verisindeki doğruluğu : {:.2f}'.format(egitim_dogruluk), 
      'Modelin test verisindeki doğruluğu   : {:.2f}'.format(test_dogruluk), sep='\n')

One-vs-rest
--------------------
Modelin eğitim verisindeki doğruluğu : 0.79
Modelin test verisindeki doğruluğu   : 0.79


In [14]:
#yukarda varsayılan onerest idi aşağıda parametreleri değiştirdik. Multinomial soft max seçildiğindeki durum.

In [15]:
log_reg_mnm = LogisticRegression(multi_class='multinomial', solver='lbfgs')
log_reg_mnm.fit(X_eğitim, y_eğitim)
egitim_dogruluk = log_reg_mnm.score(X_eğitim, y_eğitim)
test_dogruluk = log_reg_mnm.score(X_test, y_test)
print('Multinomial (Softmax)', '-'*20, 
      'Modelin eğitim verisindeki doğruluğu : {:.2f}'.format(egitim_dogruluk), 
      'Modelin test verisindeki doğruluğu   : {:.2f}'.format(test_dogruluk), sep='\n')

Multinomial (Softmax)
--------------------
Modelin eğitim verisindeki doğruluğu : 0.79
Modelin test verisindeki doğruluğu   : 0.78


In [16]:
#'fare' kolonunu ekleyince değişiklik olmadı. Bu seferde üstteki değişkenlerden 'parch' çıkarıp bakalım. 
X=df[['Pclass','Sex','Age','SibSp']]
y=df['Survived']

In [17]:
X = pd.get_dummies(data=X, drop_first=True)

In [18]:
X_eğitim, X_test, y_eğitim, y_test =  train_test_split(X, y, test_size=0.20, random_state=111)

In [19]:
log_reg = LogisticRegression()

In [20]:
log_reg.fit(X_eğitim, y_eğitim)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
egitim_dogruluk = log_reg.score(X_eğitim, y_eğitim)
test_dogruluk = log_reg.score(X_test, y_test)
print('One-vs-rest', '-'*20, 
      'Modelin eğitim verisindeki doğruluğu : {:.2f}'.format(egitim_dogruluk), 
      'Modelin test verisindeki doğruluğu   : {:.2f}'.format(test_dogruluk), sep='\n')

One-vs-rest
--------------------
Modelin eğitim verisindeki doğruluğu : 0.79
Modelin test verisindeki doğruluğu   : 0.79


In [22]:
log_reg_mnm = LogisticRegression(multi_class='multinomial', solver='lbfgs')
log_reg_mnm.fit(X_eğitim, y_eğitim)
egitim_dogruluk = log_reg_mnm.score(X_eğitim, y_eğitim)
test_dogruluk = log_reg_mnm.score(X_test, y_test)
print('Multinomial (Softmax)', '-'*20, 
      'Modelin eğitim verisindeki doğruluğu : {:.2f}'.format(egitim_dogruluk), 
      'Modelin test verisindeki doğruluğu   : {:.2f}'.format(test_dogruluk), sep='\n')

Multinomial (Softmax)
--------------------
Modelin eğitim verisindeki doğruluğu : 0.79
Modelin test verisindeki doğruluğu   : 0.77


In [23]:
#modelin son halinde aşağıdaki değişkenler kullanılıyor.
#X=df[['Pclass','Sex','Age','SibSp']]

In [32]:
#Modelimizi kullanarak nasıl tahmin yapabileceğimize bakalım. 
#Bunun için modelimizle birlikte .predict() fonksiyonunu kullanmamız yeterli olacaktır.
predictions=log_reg.predict(X_test)
print(predictions)

[1 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1
 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0
 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0]
